## TheyDo Technical Interview

### Finetune a model for:
1. summarisation
2. translation
3. sentiment analysis

Based on a dataset from [llama paper](https://arxiv.org/pdf/2302.13971.pdf)

Couldn't find a clear summarisation, translation or sentiment analysis dataset from the paper.

### Finetune Summarisation model

Which model should we go for?

<img src="cnn_dailymail_summarisation_landscape.png">

T5 is a great model for any text to text task as such a great fit for summarisation and translationg.
I have used T5 before and the small T5 can be finetuned on my Mac.

How to measure summarisation performance?

ROUGE-1 refers to the overlap of unigrams (each word) between the system and reference summaries
ROUGE-2 bigram of above
ROUGE-L Longest Common Subsequence (LCS) based score on sentences
ROUGE-LSUM Take new lines as sentence boundaries and then LCS score

---


Train eval test 5/5/5
  
***** eval metrics *****
  epoch                   =        3.0
  eval_gen_len            =       54.4
  eval_loss               =     2.4578
  eval_rouge1             =    20.1191
  eval_rouge2             =     3.2284
  eval_rougeL             =    14.6009
  eval_rougeLsum          =    19.8187
  eval_runtime            = 0:00:04.03
  eval_samples            =          5
  eval_samples_per_second =      1.238
  eval_steps_per_second   =      0.495

Train eval test 2000 500 500

***** eval metrics *****
"epoch": 3.0,
"eval_gen_len": 59.248,
"eval_loss": 2.052272081375122,
"eval_rouge1": 31.186,
"eval_rouge2": 11.7692,
"eval_rougeL": 22.3708,
"eval_rougeLsum": 28.5695,
"eval_runtime": 2077.82,
"eval_samples": 500,
"eval_samples_per_second": 0.241,
"eval_steps_per_second": 0.06




  

In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("finetuned_summarisation")
model = AutoModelForSeq2SeqLM.from_pretrained("finetuned_summarisation")


/Users/chris/work/theydo_ml_challenge/theydo/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
example = """WASHINGTON, June 7 (Reuters) - 

Ukrainians abandoned inundated homes on Wednesday as floods crested across the south after the destruction of the dam, with Russia and Ukraine trading blame for the disaster.

The World Bank will support Ukraine by conducting a rapid assessment of damage and needs after Tuesday's destruction of a huge hydroelectric dam on the front lines between Russian and Ukrainian forces, a top bank official said on Wednesday.

Anna Bjerde, the World Bank's managing director for operations, said on Twitter the destruction of the Novo Kakhovka dam had "many very serious consequences for essential service delivery and the broader environment."

Ukrainian Prime Minister Denys Shmyhal, also writing on Twitter, said he spoke with Bjerde about the impact of the dam's collapse, and she assured him the World Bank would carry out a rapid assessment of the damage and needs.

The World Bank will support Ukraine by conducting a rapid assessment of damage and needs after Tuesday's destruction of a huge hydroelectric dam on the front lines between Russian and Ukrainian forces, a top bank official said on Wednesday.

Ukrainians abandoned inundated homes on Wednesday as floods crested across the south after the destruction of the dam, with Russia and Ukraine trading blame for the disaster.

Ukraine said the deluge would leave hundreds of thousands of people without access to drinking water, swamp tens of thousands of hectares of agricultural land and turn at least 500,000 hectares deprived of irrigation into "deserts"."""

In [3]:
example = example.strip()

In [4]:
input_ids = tokenizer(f"summarize: {example}", return_tensors="pt").input_ids
outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

World Bank will support Ukraine by conducting a rapid assessment of damage and needs. Ukrainians


/Users/chris/work/theydo_ml_challenge/theydo/lib/python3.11/site-packages/transformers/generation/utils.py:1346: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [7]:
# TODO load saved models and test them

In [10]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("finetuned_de-en_translation")
model = AutoModelForSeq2SeqLM.from_pretrained("finetuned_de-en_translation")


# task_prefix = "translate German to English: "
task_prefix = ""

sentences = ["Das Haus ist wunderbar.", "Ich arbeite gerne in NYC."]

In [11]:
inputs = tokenizer([f"{task_prefix}{sentence}" for sentence in sentences], return_tensors="pt", padding=True)
outputs = model.generate(inputs["input_ids"])
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(tokenizer.decode(outputs[1], skip_special_tokens=True))

The house is wonderful.
I like working in NYC.


In [12]:
outputs[0]

tensor([    0,  1150,  1925,    56,   380, 11897,    57, 13646,  3607,  4193,
           13,  1783,    11,   523,     3,     5, 22849,     7, 13876,    16])

## There's a bug in the sentiment extraction model:

In [12]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("finetuned-imdb-bert")
model = AutoModelForSequenceClassification.from_pretrained("finetuned-imdb-bert")

In [13]:
text = "Pirates of the Caribean was a horrible movie."


In [21]:
import torch

def sent_pred(text):

    inputs = tokenizer(text, return_tensors="pt")
    
    with torch.no_grad():
    
        logits = model(**inputs).logits
    
    predicted_class_id = logits.argmax().item()
    predicted_label = model.config.id2label[predicted_class_id]
    probs = torch.nn.functional.softmax(logits)
    print(f"Predicted {predicted_label} {probs}")

sent_pred(text)

Predicted 0 tensor([[0.9987, 0.0013]])


/var/folders/6v/nj127lc1663btr53s35_n_fh0000gn/T/ipykernel_71500/4045664043.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = torch.nn.functional.softmax(logits)


In [23]:
sent_pred("Inception was the worst")

Predicted 0 tensor([[0.9903, 0.0097]])


/var/folders/6v/nj127lc1663btr53s35_n_fh0000gn/T/ipykernel_71500/4045664043.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = torch.nn.functional.softmax(logits)


0

## Compare OpenAI and finetuned models:

In [30]:
import pandas as pd

cnn_test = pd.read_csv("data/cnn_dailymail/test.csv")


In [31]:
cnn_test

,id,article,highlights
0,92c514c913c0bdfe25341af9fd72b29db544099b,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...
1,2003841c7dc0e7c5b1a248f9cd536d727f27a45a,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...
2,91b7d2311527f5c2b63a65ca98d21d9c92485149,Dougie Freedman is on the verge of agreeing a ...,Nottingham Forest are close to extending Dougi...
3,caabf9cbdf96eb1410295a673e953d304391bfbb,Liverpool target Neto is also wanted by PSG an...,Fiorentina goalkeeper Neto has been linked wit...
4,3da746a7d9afcaa659088c8366ef6347fe6b53ea,Bruce Jenner will break his silence in a two-h...,"Tell-all interview with the reality TV star, 6..."
...,...,...,...
11485,ed8674cc15b29a87d8df8de1efee353d71122272,Our young Earth may have collided with a body ...,Oxford scientists say a Mercury-like body stru...
11486,2f58d1a99e9c47914e4b1c31613e3a041cd9011e,A man facing trial for helping his former love...,Man accused of helping former lover kill woman...
11487,411f6d57825161c3a037b4742baccd6cd227c0c3,A dozen or more metal implements are arranged ...,Marianne Power tried the tuning fork facial at...
11488,b5683ef8342056b17b068e0d59bdbe87e3fe44ea,Brook Lopez dominated twin brother Robin with ...,Brooklyn Nets beat the Portland Trail Blazers ...
